importation des bibliothèques nécessaires et chargement de fichier à etudier

In [1]:
import os 
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
import datetime
base='..\\data\\nettoyer\\Online_Retail_a_analyser.csv'
df_a_analyser= pd.read_csv(base,parse_dates=['InvoiceDate'])

preparation du dataframe en calculant le revenu

In [2]:
df_a_analyser['Revenue']=df_a_analyser['UnitPrice']*df_a_analyser['Quantity']

détermination de la date de la dernière commande

In [3]:
date_max= df_a_analyser['InvoiceDate'].max() +pd.Timedelta(days=1)

creation du data_frame score R:Recency F :Frequency M:Monetary

In [4]:
rfm = df_a_analyser.groupby('CustomerID').agg({
'InvoiceDate': lambda x: (date_max- x.max()).days,
'InvoiceNo': 'nunique',
'Revenue': 'sum'
}).reset_index()         
rfm.rename(columns={'InvoiceDate':'Recency','InvoiceNo':'Frequency','Revenue':'Monetary'}, inplace=True)
print(rfm.head())

   CustomerID  Recency  Frequency  Monetary
0     12346.0      326          1  77183.60
1     12347.0        2          7   4310.00
2     12348.0       75          4   1797.24
3     12349.0       19          1   1757.55
4     12350.0      310          1    334.40


La segmentation RFM repose sur l'attribution de scores selons trois critères:
-La Récence : où un score élevé valorise la proximité temporelle du dernière achat.
-La Fréquence:récompensant le nombre élevé de transactions.
-Le Montant :distinguant les clients ayant généré le chiffre d'affaire le lus conséquent.


In [5]:
rfm['R_Score'] = pd.qcut(rfm['Recency'], 5, labels=[5, 4, 3, 2, 1], duplicates='drop')  


rfm['F_Score'] = pd.qcut(rfm['Frequency'].rank(method='first'), 5, labels=[1, 2, 3, 4, 5], duplicates='drop')
rfm['M_Score'] = pd.qcut(rfm['Monetary'].rank(method='first'), 5, labels=[1, 2, 3, 4, 5], duplicates='drop')

affichage du score sous forme de caratère string (lettres)

In [9]:
rfm['RFM_Score'] = rfm['R_Score'].astype(str) + \
                      rfm['F_Score'].astype(str) + \
                      rfm['M_Score'].astype(str)

print(rfm[['CustomerID','RFM_Score']].head().to_string(index=False))
rfm['R_Score']=rfm['R_Score'].astype(str)
rfm['F_Score']=rfm['F_Score'].astype(str)
rfm['M_Score']=rfm['M_Score'].astype(str)


 CustomerID RFM_Score
    12346.0       115
    12347.0       555
    12348.0       244
    12349.0       414
    12350.0       112


Dfinir les conditions de la  segmentation finale

In [16]:
conditions = [
    # Champions : Récence élevée (4 ou 5) ET Fréquence/Montant élevés (4 ou 5)
    (rfm['R_Score'].isin(['4', '5'])) & (rfm['F_Score'].isin(['4', '5'])) & (rfm['M_Score'].isin(['4', '5'])),
    # Clients Fidèles : Récence moyenne (2 ou 3) ET Fréquence/Montant élevés (4 ou 5)
    (rfm['R_Score'].isin(['2', '3','4'])) & (rfm['F_Score'].isin(['3','4', '5'])),
    # Clients En Danger : Récence faible (1 ou 2) mais historiquement bons (F & M élevés)
    (rfm['R_Score'].isin(['1', '2'])) & (rfm['F_Score'].isin(['4', '5'])),
    # Clients Perdus : Tout est faible
    (rfm['R_Score'].isin(['1', '2'])) & (rfm['F_Score'].isin(['1', '2'])),
    (rfm['R_Score'].isin(['4', '5'])) & (rfm['F_Score'].isin(['1']))
]

choix_segments = [
    'Meilleurs Clients (Champions)',
    'Clients Fidèles',
    'Clients En Danger',
    'Clients Perdus',
    'Nouveau client'
]
# 6. Application des règles (le reste est classé 'Autres')
rfm['Segment'] = np.select(conditions, choix_segments, default='Autres')
rfm


,CustomerID,Recency,Frequency,Monetary,R_Score,F_Score,M_Score,RFM_Score,Segment
0,12346.0,326,1,77183.60,1,1,5,115,Clients Perdus
1,12347.0,2,7,4310.00,5,5,5,555,Meilleurs Clients (Champions)
2,12348.0,75,4,1797.24,2,4,4,244,Clients Fidèles
3,12349.0,19,1,1757.55,4,1,4,414,Nouveau client
4,12350.0,310,1,334.40,1,1,2,112,Clients Perdus
...,...,...,...,...,...,...,...,...,...
4334,18280.0,278,1,180.60,1,2,1,121,Clients Perdus
4335,18281.0,181,1,80.82,1,2,1,121,Clients Perdus
4336,18282.0,8,2,178.05,5,3,1,531,Autres
4337,18283.0,4,16,2094.88,5,5,5,555,Meilleurs Clients (Champions)


In [27]:

pourcentage=rfm.groupby(['Segment']).size().reset_index(name='nombre')
somme=pourcentage['nombre'].sum()
somme
pourcentage['pourcent']=((pourcentage['nombre']/somme)*100 ).map('{:.2f}%'.format)  # afficher deux chiffres apres


calcule revenue par segment 

In [36]:
pourcentage_rev=rfm.groupby('Segment')['Monetary'].sum().reset_index(name='total_revenue')
somme_rev=pourcentage_rev['total_revenue'].sum()
pourcentage_rev['pourcent']=((pourcentage_rev['total_revenue']/somme_rev)*100 ).map('{:.2f}%'.format)
pourcentage_rev


,Segment,total_revenue,pourcent
0,Autres,751435.400,8.43%
1,Clients En Danger,71393.430,0.80%
2,Clients Fidèles,1708344.712,19.17%
3,Clients Perdus,519408.572,5.83%
4,Meilleurs Clients (Champions),5809341.070,65.19%
5,Nouveau client,51484.720,0.58%
